In [6]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import warnings
from operator import itemgetter

from langchain_community.llms import Together
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_together import TogetherEmbeddings
from load import load_and_clean_html_texts
from langchain.docstore.document import Document

In [5]:
# Load, chunk and index the contents of the blog.
docs = load_and_clean_html_texts("AAPL")



Issue with /Users/kaushikarcot/Documents/GT/Summer/FinTech/nlp/sec-edgar-filings/AAPL/10-K/0001193125-14-383437


AttributeError: 'str' object has no attribute 'page_content'

In [7]:
final_docs = []

for doc in docs:
    f =  Document(page_content=doc, metadata={"source": "local"})
    final_docs.append(f)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(final_docs)

In [8]:
splits[0].page_content

'-----BEGIN PRIVACY-ENHANCED MESSAGE-----\nProc-Type: 2001,MIC-CLEAR\nOriginator-Name: webmaster@www.sec.gov\nOriginator-Key-Asymmetric:\n MFgwCgYEVQgBAQICAf8DSgAwRwJAW2sNKK9AVtBzYZmr6aGjlWyK3XmZv3dTINen\n TWSM7vrzLADbmYQaionwg5sDW3P6oaM5D3tdezXMm7z1T+B+twIDAQAB\nMIC-Info: RSA-MD5,RSA,\n MjhsC9mG2K0EvUka1yxHNJkRoDsL7RoD49ttKgFrzkpRz+LMwhc5oFefyjPoytyg\n ARYgB3AmlXI2xC09czbvbA== 0001193125-08-224958.txt : 20081105 0001193125-08-224958.hdr.sgml : 20081105 20081105061623\nACCESSION NUMBER:\t\t0001193125-08-224958\nCONFORMED SUBMISSION TYPE:\t10-K\nPUBLIC DOCUMENT COUNT:\t\t7\nCONFORMED PERIOD OF REPORT:\t20080927\nFILED AS OF DATE:\t\t20081105\nDATE AS OF CHANGE:\t\t20081105\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tAPPLE INC\n\t\tCENTRAL INDEX KEY:\t\t\t0000320193\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tELECTRONIC COMPUTERS [3571]\n\t\tIRS NUMBER:\t\t\t\t942404110\n\t\tSTATE OF INCORPORATION:\t\t\tCA\n\t\tFISCAL YEAR END:\t\t\t0930'

In [12]:
len(splits)

133225

In [ ]:
warnings.filterwarnings("ignore")

vectorstore = FAISS.from_texts(
    [x.page_content for x in splits],
    TogetherEmbeddings(model="togethercomputer/m2-bert-80M-8k-retrieval"),
)

print("Indexing Done")

retriever = vectorstore.as_retriever()

model = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1"
)

# Provide a template following the LLM's original chat template.
template = """<s>[INST] Answer the question in a simple sentence based only on the following context:
{context}

Question: {question} [/INST] 
"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

input_query = "What are some insights you can draw about Apple from this data?"
output = chain.invoke(input_query)

print(output)